In [1]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

In [2]:
%aimport daq
DAQ = daq.Daq()
DAQ.connect_instr('TDS2024C', 1)
DAQ.connect_instr('KA3005P', 'ttyUSB0')
DAQ.connect_instr('Camera', 17583372)
DAQ.connect_instr('Camera', 17529184)
#DAQ.connect_instr('HR4000', 'michaelito')

Power supply ID: KORADKA3005PV2.0
Device ttyUSB0 successfully connected and process started.
Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Device C046401 successfully connected and process started.
Camera ID: Blackfly BFLY-PGE-50S5M,17529184,FV:1.53.3.0
Camera ID: Blackfly BFLY-PGE-31S4M,17583372,FV:1.61.3.0
Device 17529184 successfully connected and process started.
Device 17583372 successfully connected and process started.


In [3]:
v = np.linspace(0, 20, 1000)
DAQ.send_command(DAQ.command_queue[17583372], 'set_frame_rate', (10,))
DAQ.send_command(DAQ.command_queue[17529184], 'set_frame_rate', (10,))
for i in range(1000):
    DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_voltage', (v[i],))
DAQ.send_command(DAQ.command_queue[17583372], 'save_stream')
DAQ.send_command(DAQ.command_queue[17529184], 'save_stream')
#time.sleep(4)
#DAQ.send_command(DAQ.command_queue[17583372], 'stop_capture')
#DAQ.send_command(DAQ.command_queue[17529184], 'stop_capture')

In [4]:
DAQ.send_command(DAQ.command_queue[17583372], 'stop_capture')
DAQ.send_command(DAQ.command_queue[17529184], 'stop_capture')

In [4]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_record_voltage', (15.67,))
DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform')

In [5]:
names = []
for name in DAQ.instr:
    names.append(name)
for name in names:
    DAQ.disconnect_instr(name)